In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
# 1. Load and prepare your data
DATA_PATH = 'german_credit_data_cleaned.csv'
df = pd.read_csv(DATA_PATH)

y = df['Risk_good'].values
X_full = df.drop(columns=['Risk_good']) 

# extract sensitive attribute array
sensitive = df['Sex_male'].values


In [3]:

# split data
def split_data(X_full, y, sensitive, test_size=0.2, random_state=200):
    X_train, X_test, y_train, y_test, sens_train, sens_test = \
        train_test_split(
            X_full.drop(columns=['Sex_male']).values,
            y,
            sensitive,
            test_size=test_size,
            random_state=random_state
        )
    return X_train, X_test, y_train, y_test, sens_train, sens_test

X_train, X_test, y_train, y_test, sens_train, sens_test = split_data(X_full, y, sensitive)

# prepare DMatrix

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)



In [23]:
# 2. Define factory for custom objective using closure to capture sensitive array

def make_custom_obj(sens_array, alpha=0.5):
    def custom_obj(preds, dmatrix):
        # raw margins -> probabilities
        labels = dmatrix.get_label()
        preds_prob = 1.0 / (1.0 + np.exp(-preds))

        # 1. log-loss gradients
        grad_log = preds_prob - labels
        hess_log = preds_prob * (1.0 - preds_prob)

        # 2. fairness gradients (DPD)
        sens = sens_array  # closure-captured train/test sens aligned to DMatrix
        mask_p = (sens == 1)
        mask_u = (sens == 0)
        n_p, n_u = mask_p.sum(), mask_u.sum()
        mean_p = preds_prob[mask_p].mean()
        mean_u = preds_prob[mask_u].mean()
        diff = mean_p - mean_u
        sign = np.sign(diff)

        sigmoid_deriv = preds_prob * (1.0 - preds_prob)
        grad_fair = np.zeros_like(preds)
        grad_fair[mask_p] =  sign * (1.0/n_p) * sigmoid_deriv[mask_p]
        grad_fair[mask_u] = -sign * (1.0/n_u) * sigmoid_deriv[mask_u]
        hess_fair = np.zeros_like(preds)

        grad = alpha*grad_log + (1-alpha)*grad_fair
        hess = alpha*hess_log + (1-alpha)*hess_fair
        return grad, hess
    return custom_obj


In [5]:

# Loop over alpha values and train separate models
alpha_list = [i/10 for i in range(1, 11)]  # 0.1 to 1.0
results = []
for alpha in alpha_list:
    print(f"Training with alpha = {alpha}")
    # create custom objective for this alpha
    train_obj = make_custom_obj(sens_train, alpha=alpha)
    # train model
    params = {'max_depth': 4, 'eta': 0.1, 'verbosity': 0}
    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=100,
        obj=train_obj,
        evals=[(dtrain, 'train'), (dtest, 'test')],
        early_stopping_rounds=10,
        verbose_eval=False
    )
    # evaluate on test set
    pred_prob = bst.predict(dtest)
    pred = (pred_prob > 0.5).astype(int)
    acc = accuracy_score(y_test, pred)
    dpd_val = abs(pred[sens_test==1].mean() - pred[sens_test==0].mean())
    # print(f" -> Acc: {acc:.4f}, DPD: {dpd_val:.4f}")
    results.append({'alpha': alpha, 'accuracy': acc, 'dpd': dpd_val})



Training with alpha = 0.1
Training with alpha = 0.2
Training with alpha = 0.3
Training with alpha = 0.4
Training with alpha = 0.5
Training with alpha = 0.6
Training with alpha = 0.7
Training with alpha = 0.8
Training with alpha = 0.9
Training with alpha = 1.0


In [6]:
# Summarize results
print("Summary of results for different alpha values:")
alpha_vals = []
acc_vals = []
dpd_vals = []
for res in results:
    alpha_vals.append(res['alpha'])
    acc_vals.append(res['accuracy'])
    dpd_vals.append(res['dpd'])
    print(f"alpha={res['alpha']}: acc={res['accuracy']:.4f}, dpd={res['dpd']:.4f}")



Summary of results for different alpha values:
alpha=0.1: acc=0.6893, dpd=0.0094
alpha=0.2: acc=0.6714, dpd=0.0366
alpha=0.3: acc=0.6821, dpd=0.0151
alpha=0.4: acc=0.6929, dpd=0.0228
alpha=0.5: acc=0.6964, dpd=0.0118
alpha=0.6: acc=0.6929, dpd=0.0065
alpha=0.7: acc=0.6964, dpd=0.0118
alpha=0.8: acc=0.6893, dpd=0.0012
alpha=0.9: acc=0.6929, dpd=0.0122
alpha=1.0: acc=0.6893, dpd=0.0012


In [7]:
# 6. Identify Pareto-optimal points and create interactive plot with Plotly
import pandas as pd
import plotly.express as px

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient

# compute Pareto front mask
efficient_mask = is_pareto_efficient(acc_vals, dpd_vals)

# prepare DataFrame for plotting
plot_df = pd.DataFrame({
    'DPD': dpd_vals,
    'Accuracy': acc_vals,
    'alpha': alpha_vals,
    'Pareto': ['Yes' if m else 'No' for m in efficient_mask]
})

plot_df['Dominated'] = ['No' if m else 'Yes' for m in efficient_mask]

# Plotly: color Pareto points by alpha, dominated in gray
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True, 'Dominated': True},
    title='Interactive Pareto and Dominated Points: DPD vs. Accuracy'
)
# update marker styling: dominated points gray
fig.update_traces(
    selector=dict(),
    marker=dict(line_width=0)
)
# add dominated points as separate layer
dominated_df = plot_df[plot_df['Dominated']=='Yes']
fig.add_scatter(
    x=dominated_df['DPD'],
    y=dominated_df['Accuracy'],
    mode='markers',
    marker=dict(color='lightgray', size=10),
    hoverinfo='skip',
    name='Dominated'
)
fig.show()

In [14]:
import random

alpha_list = []
for i in range(1, 101):
    alpha_list.append(i/100)

from itertools import product
from sklearn.model_selection import ParameterGrid

# your expanded param_grid
param_grid = {
    'n_estimators':     [50, 100, 200],
    'learning_rate':    [0.01, 0.05, 0.1],
    'max_depth':        [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'gamma':            [0, 0.1, 0.2],
    'subsample':        [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha':        [0, 0.01, 0.1],
    'reg_lambda':       [1, 1.5, 2],
    'scale_pos_weight': [1, 3, 5],
    'alpha':            alpha_list  
}

all_params = list(ParameterGrid(param_grid))

# sample 100 (or fewer if grid <100)
sampled_params = random.sample(all_params, min(100, len(all_params)))

results = []
for params in sampled_params:
    alpha = params.pop('alpha')
    print(f"Training with params: {params}, alpha={alpha}")
    train_obj = make_custom_obj(sens_train, alpha)
    bst = xgb.train(
        params,
        dtrain,
        num_boost_round=100,
        obj=train_obj,
        evals=[(dtrain, 'train'), (dtest, 'test')],
        early_stopping_rounds=10,
        verbose_eval=False
    )
    pred_prob = bst.predict(dtest)
    pred = (pred_prob > 0.5).astype(int)
    acc     = accuracy_score(y_test, pred)
    dpd_val = abs(pred[sens_test==1].mean() - pred[sens_test==0].mean())
    results.append({ **params, 'alpha': alpha, 'accuracy': acc, 'dpd': dpd_val })

Training with params: {'colsample_bytree': 1.0, 'gamma': 0.1, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 100, 'reg_alpha': 0.1, 'reg_lambda': 1, 'scale_pos_weight': 3, 'subsample': 1.0}, alpha=0.1
Training with params: {'colsample_bytree': 1.0, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 5, 'subsample': 0.8}, alpha=0.57
Training with params: {'colsample_bytree': 1.0, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 50, 'reg_alpha': 0, 'reg_lambda': 2, 'scale_pos_weight': 1, 'subsample': 0.8}, alpha=0.91
Training with params: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'reg_alpha': 0.01, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.6}, alpha=0.61
Training with params: {'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate':

In [17]:
# Summarize results
print("Summary of results for different alpha values:")
alpha_vals = []
acc_vals = []
dpd_vals = []
max_depth_vals = []
for res in results:
    # print("res:",res)
    alpha_vals.append(res['alpha'])
    acc_vals.append(res['accuracy'])
    dpd_vals.append(res['dpd'])
    max_depth_vals.append(res['max_depth'])
    # eta_vals.append(res['eta:'])
    print(f"max_depth={res['max_depth']}, acc={res['accuracy']:.4f}, dpd={res['dpd']:.4f}")

Summary of results for different alpha values:
max_depth=3, acc=0.6893, dpd=0.0501
max_depth=7, acc=0.6893, dpd=0.0379
max_depth=3, acc=0.6750, dpd=0.0427
max_depth=3, acc=0.6786, dpd=0.0098
max_depth=5, acc=0.6821, dpd=0.0037
max_depth=5, acc=0.6929, dpd=0.0122
max_depth=3, acc=0.6893, dpd=0.0256
max_depth=3, acc=0.6893, dpd=0.0102
max_depth=3, acc=0.6786, dpd=0.0147
max_depth=3, acc=0.5357, dpd=0.0000
max_depth=7, acc=0.6893, dpd=0.0419
max_depth=5, acc=0.6857, dpd=0.0252
max_depth=7, acc=0.7000, dpd=0.0163
max_depth=3, acc=0.6714, dpd=0.0252
max_depth=7, acc=0.6893, dpd=0.0126
max_depth=7, acc=0.6964, dpd=0.0167
max_depth=3, acc=0.6893, dpd=0.0501
max_depth=7, acc=0.7071, dpd=0.0049
max_depth=7, acc=0.6929, dpd=0.0114
max_depth=5, acc=0.6786, dpd=0.0016
max_depth=5, acc=0.6857, dpd=0.0147
max_depth=3, acc=0.6750, dpd=0.0151
max_depth=5, acc=0.6821, dpd=0.0094
max_depth=3, acc=0.6750, dpd=0.0370
max_depth=3, acc=0.6893, dpd=0.0501
max_depth=3, acc=0.6929, dpd=0.0269
max_depth=5, acc=

In [19]:
# 6. Identify Pareto-optimal points and create interactive plot with Plotly
import pandas as pd
import plotly.express as px

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient

# compute Pareto front mask
efficient_mask = is_pareto_efficient(acc_vals, dpd_vals)

# prepare DataFrame for plotting
plot_df = pd.DataFrame({
    'DPD': dpd_vals,
    'Accuracy': acc_vals,
    'alpha': alpha_vals,
    'max_depth': max_depth_vals,
    # 'eta': eta_vals,    
    'Pareto': ['Yes' if m else 'No' for m in efficient_mask]
})

plot_df['Dominated'] = ['No' if m else 'Yes' for m in efficient_mask]

# Plotly: color Pareto points by alpha, dominated in gray
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True, 'Dominated': True, 'max_depth': True},
    title='Interactive Pareto and Dominated Points: DPD vs. Accuracy'
)
# update marker styling: dominated points gray
fig.update_traces(
    selector=dict(),
    marker=dict(line_width=0)
)
# add dominated points as separate layer
dominated_df = plot_df[plot_df['Dominated']=='Yes']
fig.add_scatter(
    x=dominated_df['DPD'],
    y=dominated_df['Accuracy'],
    mode='markers',
    marker=dict(color='lightgray', size=10),
    hoverinfo='skip',
    name='Dominated'
)
fig.show()

In [33]:
! pip install optuna 

  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-win_amd64.whl.metadata (4.1 kB)
Using cached alembic-1.15.2-py3-none-any.whl (231 kB)
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/1.9 MB 1.5 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/1.9 MB 1.0 MB/s eta 0:00:02
   --------------------- ------------------ 1.0/1.9 MB 1.1 MB/s eta 0:00:01
   --------------------------- ------------ 1.3/1.9 MB 1.2 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/1.9 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 1.3 MB/s eta 0:00:00
Using cache


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
